In [1]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline 

# Dados coletados


In [2]:
#Tempo de subida t_r (matrix que acomoda todos os conjuntos de dados em uma única variável)
t_r = []

#Primeiro conjunto de dados
t_r1 = np.array([0.47,1.00,0.59,1.78,0.94,0.75,1.19,0.47]) #s
t_r.append(t_r1)

#Segundo conjunto de dados
#t_r2 = np.array([])
#t_r.append(t_r2)





#Tempo de queda (matrix que acomoda todos os conjuntos de dados em uma única variável)
t_f = []

#Primeiro conjunto de dados
t_f1 = np.array([0.47,0.47,1.16,0.47,1.09,1.10,0.91,1.18,1.16]) #s
t_f.append(t_f1)

#Segundo conjunto de dados
#t_f2 = np.array([])
#t_f.append(t_f2)


#Separação das placas do capacitor e sua incerteza instrumental:
d = 8.75e-3 #m
sigma_d = 5e-5 #m

#Diferença de potencial aplicada nas placas e incerteza instrumental
delta_V = 202 #V
sigma_V = 0.5 #V


#Os intervalos de tempo foram medidos com um cronômetro de incerteza
sigma_t_inst = 0.005 #s

# Incerteza do tempo

Nós temos uma incerteza instrumental e também estatística no tempo medido. Tendo isto em vista devemos unir as duas da seguinte forma:

$$\sigma_\bar{t} = \sqrt{\sigma_{inst}^2 + \sigma_{est}^2}$$

Onde $\sigma_{est}$ é o desvio padrão da média.

In [3]:
#Incerteza do tempo de queda
sigma_t_f = []

for t in t_f:
    aux = np.sqrt(sigma_t_inst**2 + (np.std(t)/(len(t)-1))**2 )
    aux = float('{:g}'.format(float('{:.2g}'.format(aux))))
    sigma_t_f.append(aux)
print(sigma_t_f)

#Incerteza do tempo de subida
sigma_t_r = []

for t in t_r:
    aux = np.sqrt(sigma_t_inst**2 + (np.std(t)/(len(t)-1))**2 )
    aux = float('{:g}'.format(float('{:.2g}'.format(aux))))
    sigma_t_r.append(aux)
print(sigma_t_r)

[0.039]
[0.059]


# Calculando as velocidades


Seja $x$ o comprimento da menor divisão da grade, então a velocidade da gota pode ser calculculada aproximadamente como:

$$v = \frac{x}{\bar{t}}$$

Onde $\bar{t}$ é o tempo médio

In [11]:
#Tamanho do intervalo da grande:
x = 5e-4 #m

#Velocidade de subida
v_r = []
for t in t_r:
    v_r.append(x/np.mean(t)) #m/s

#Velocidade de queda
v_f = []
for t in t_f:
    v_f.append(x/np.mean(t)) #m/s

# Calculando a incerteza das velocidades

A incerteza da velocidade será dada pela seguinte expressão:

$$\sigma_v = \sqrt{\left(\frac{\partial}{\partial \bar{t}}v\right)^2\sigma_\bar{t}^2} = \bigg|\frac{\partial}{\partial \bar{t}}v\bigg|\sigma_\bar{t}$$

Utilizando a expressão de $v$:

$$\sigma_v  = x\bigg|\frac{\partial}{\partial \bar{t}}\frac{1}{\bar{t}}\bigg|\sigma_\bar{t} = \frac{x}{\bar{t}^2}\sigma_{\bar{t}}$$

In [5]:
#Incerteza da velocidade de subida
sigma_v_r = []
for i,t in enumerate(t_r):
    aux = x/(np.mean(t)**2) * sigma_t_r[i]
    aux = float('{:g}'.format(float('{:.2g}'.format(aux))))
    sigma_v_r.append(aux)
print(sigma_v_r)

#Incerteza da velocidade de queda
sigma_v_f = []
for i,t in enumerate(t_f):
    aux = x/(np.mean(t)**2) * sigma_t_f[i]
    aux = float('{:g}'.format(float('{:.2g}'.format(aux))))
    sigma_v_f.append(aux)
print(sigma_v_f)
    

[3.7e-05]
[2.5e-05]


# Calculando a carga

$$q = \frac{4}{3}\pi \left[\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}} - \frac{b}{2p}\right]^3 \frac{\rho g d}{V} \frac{(v_f + v_r)}{v_f}$$

In [15]:
# Constantes que utilizaremos:

b = 8.2e-3 #Pa m

#Densidade do óleo
rho = 886 #kg/m^3

#Aceleração da gravidade
g = 9.8 #m/s

#Viscodidade do ar a uma temperatura de aproximadamente 26 graus Celsius
eta = 1.8520e-5 #Nsm^-2

#Pressão barométrica
p = 101325 #Pa


#Calculando a carga com a equação corrigida para a queda com campo elétrico ligado (Ignorar)
#q = 4/3 * np.pi * (np.sqrt((b/(2*p))**2 + 9*eta*np.array(v_f)/(2*rho*g)) - b/(2*p))**3 * rho * g * d/delta_V * (np.array(v_f) - np.array(v_r))/(np.array(v_f) + np.array(v_r)) 

#Calculando a carga uilizando a equação derivada no roteiro
q = []
for i, vf in enumerate(v_f):
    charge = 4/3 * np.pi * (np.sqrt((b/(2*p))**2 + 9*eta*vf/(2*rho*g)) - b/(2*p))**3 * rho * g * d/delta_V * (vf + v_r[i])/(vf)
    q.append(charge)

print('Carga =',q, 'C')



Carga = [3.7262705585606456e-17] C


# Calculando a incerteza da carga

Propagando a incerteza da velocidade de queda, velocidade de subida, tensão e serparação da placa dos capacitores para a carga:

$$\sigma_q = \sqrt{\left(\frac{\partial q}{\partial v_f}\right)^2\sigma_{v_f}^2 + \left(\frac{\partial q}{\partial v_r}\right)^2\sigma_{v_r}^2 + \left(\frac{\partial q}{\partial V}\right)^2\sigma_{V}^2 + \left(\frac{\partial q}{\partial d}\right)^2\sigma_{d}^2}$$

Utilizando a expressão de $q$ nós obtemos que:

$$\frac{\partial q}{\partial v_f} = \frac{4}{3}\pi\frac{\rho g d}{V}\left[\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}} - \frac{b}{2p}\right]^2\left[\frac{27}{4}\frac{\eta}{\rho g}\frac{1}{\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}}}\frac{(v_f + v_i)}{v_f} - \left(\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}} - \frac{b}{2p}\right) \frac{v_r}{v_f^2} \right]$$

$$\frac{\partial q }{\partial v_r} = \frac{4}{3}\pi \frac{\rho g d}{V}\left[\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}} - \frac{b}{2p}\right]^3 \frac{1}{v_f}$$

$$\frac{\partial q}{\partial V} = -\frac{4}{3}\pi \frac{\rho g d}{V^2}\left[\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}} - \frac{b}{2p}\right]^3\frac{(v_f + v_r)}{v_f}$$

$$\frac{\partial q}{\partial d} = \frac{4}{3}\pi \frac{\rho g }{V}\left[\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}} - \frac{b}{2p}\right]^3\frac{(v_f + v_r)}{v_f}$$


Assim:

$$\sigma_q = q\sqrt{\left[\frac{27}{4}\frac{\eta}{\rho g}\frac{1}{\left(\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}} - \frac{b}{2p}\right)\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9\eta v_f}{2\rho g}}} -  \frac{v_r}{v_f(v_f + v_r)} \right]^2 \sigma_{v_f}^2 + \left[\frac{\sigma_{v_r}}{(v_f + v_r)}\right]^2 + \left[\frac{\sigma_d}{d}\right]^2 + \left[\frac{\sigma_V}{V}\right]^2}$$

In [14]:
#Devido ao tamanho da expressão, vamos calculá-la em blocos

sigma_q = []

for i,vf in enumerate(v_f):
    #Fazendo os cálculos 
    sqrt = np.sqrt((b/(2*p))**2 + 9*eta*vf/(2*rho*g))
    x_1 = sigma_v_f[i]*(27/4 * eta/(rho * g) * 1/((sqrt - b/(2*p))*sqrt) - v_r[i]/(vf*(vf + v_r[i])))
    x_2 = sigma_v_r[i]/(vf + v_r[i])
    x_3 = sigma_d/d
    x_4 = sigma_V/delta_V
    aux = q[i]*np.sqrt(x_1**2 + x_2**2 + x_3**2 + x_4**2)
    aux = float('{:g}'.format(float('{:.2g}'.format(aux))))
    
    #Armazenando a incerteza para cada valor de carga
    sigma_q.append(aux)
    
print(sigma_q)

[5.2e-21]
